In [1]:
import pandas as pd
import numpy as np
import copy 

from pandas_profiling import ProfileReport
from datetime import datetime, timedelta
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB 
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

def _encode(df, labels, label):
    """
        Sempre: 
            0-HODL
            1-BUY
            2-SELL
    """
    df[label] = 0
    df.loc[df[labels[0]] == 1, label] = 1
    df.loc[df[labels[1]] == 1, label] = 2
    return df
def _final_encode(df, labels, label):
    """
        Sempre: 
            0-HODL
            1-BUY
            2-SELL
        no caso do final, ação = 1; ver nos indicadores e aplicar a ação escolhida
    """
    df[label] = 0
    df[label] = df[labels].sum(axis=1)
    df[label] = [ 1 if x.astype(int) > 0 else 0 for x in df[label].values ]
    return df
def _filter_Train_n_Test(df):
    train = df.loc[df.Data < pd.to_datetime("01/02/2018", format="%d/%m/%Y")]
    test = df.loc[df.Data >= pd.to_datetime("01/02/2018", format="%d/%m/%Y")]
    return train, test
def _preprocess(df, X_labels, y_label):
    train, test = _filter_Train_n_Test(df)
    X_train = train[X_labels]
    y_train = train[y_label]
    X_test = test[X_labels]
    y_test = test[y_label]
    scaler = MinMaxScaler()
    # Prevenindo data leakage
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test, scaler

def _apply_gaussianNB(df, features, label, silent=False):
    """
    Basea-se na suposição de que os dados seguem a distribuição normal para a predição da probabilidade a priori.
    As features devem ter valores contínuos (caso dos valores de preço!)
    """
    model = GaussianNB()
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Gaussian Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_complementNB(df, features, label, silent=False, _alpha = 1):
    """
    Seguimos a distribuição normal, mas generalizamos a predição calculando a probabilidade do item pertencer a todas as classes.
    (calculamos a probabilidade do item não pertencer a cada classe, selecionamos o menor valor, tendo em vista que calculamos a probabilidade de não ser da classe em cálculo)
    Útil para datasets desbalanceados!
    """
    model = ComplementNB(alpha=_alpha)
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Complement Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_multinomialNB(df, features, label, silent=False, _alpha = 1):
    """
    Esse algoritmo usa os dados em uma distribuição multinomial, que é uma generalização da distribuição binomial. 
    Essa distribuição é parametrizada por vetores θyi=(θy1,…,θyn), θyi é a probabilidade do evento i ocorrer, dado que a classe é y
    """
    model = MultinomialNB(alpha=_alpha)
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Multinomial Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_model(df, model, scaler, features, label, decisions):
    """
    Aplica as decisões dos primeiros modelos, para avaliação de como ponderar para uma gestão de risco baseada em resultados anteriores
    """
   # Variáveis de apoio
    trades = 0 if len(decisions)==0 else len(decisions) - 1

    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)

    # Separamos as colunas para a análise
    cols = copy.deepcopy(features)
    cols.append(label)
    cols.append('Data')

    # Pegamos do df as colunas
    features_df = df[cols]

    # para cada linha, aplicamos a decisão do modelo
    for i, item in df.iterrows():
        row_features = scaler.transform(pd.DataFrame(item[features]).T)
        _data = item['Data']
        if (trades > 0 and decisions.loc[trades-1, 'classifier'] == label):
            _last_decision_date = decisions.iloc[trades-1].Data
            _last_model_decision = decisions.iloc[trades-1].model_decision
            _last_real_decision = decisions.iloc[trades-1].real_decision
        else:
            _last_decision_date = pd.to_datetime('01/01/1950')
            _last_model_decision = 2
            _last_real_decision = 2
        _classifier = label
        _real_decision = item[label]
        _model_decision = model.predict(row_features)[0] if label != 'divergency_decisor' else item[label] # No caso do decisor de divergência vamos usar o próprio dado real
        _value_fechamento = item['Fechamento']
        #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
        #if ((_model_decision != 0) or (_real_decision != 0)):
        if ((_data >= _last_decision_date + timedelta(days=wait_time))):
            if ((_model_decision != 0 and _model_decision != _last_model_decision) or (_real_decision != 0 and _real_decision != _last_real_decision)):
            # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                decisions.loc[trades,'Data'] = _data
                decisions.loc[trades,'classifier'] = _classifier
                decisions.loc[trades,'real_decision'] = _real_decision
                decisions.loc[trades,'model_decision'] = _model_decision
                decisions.loc[trades,'value_fechamento'] = _value_fechamento
                trades += 1
    return decisions

def _apply_decisions(model_decisions, classifier, money_start, decision_weight, n):
    """
    Aqui, aplicamos as decisões dos modelos sem uma gestão de risco, para gerar os pesos de ponderação para a gestão de risco;
    "All in, All out!", ou seja, compramos tudo ou vendemos tudo!
    """
    last_decision = 2
    money_atual = money_start
    quant_comprada = 0
    print("--------------------------------------------------------")
    print(classifier)
    print("--------------------------------------------------------")
    for i, row in model_decisions.iterrows():
        if (row.model_decision != last_decision and row.model_decision !=0):
            last_decision = row.model_decision
            if (last_decision == 1):
                quant_comprada = float(money_atual) / float(row.value_fechamento)
                money_atual = float(money_atual) - (float(quant_comprada) * float(row.value_fechamento))
                print("Decisão de compra -")
                print('Valor: {}'.format(str(row.value_fechamento)))
                print('Quantidade comprada: {}'.format(str(quant_comprada)))
                print("Money atual: {}".format(str(money_atual)))
            elif (last_decision == 2):
                money_comprado = float(quant_comprada) * float(row.value_fechamento)
                money_atual = float(money_atual) + float(money_comprado)
                print("Decisão de venda -")
                print('Valor: {}'.format(str(row.value_fechamento)))
                print('Quantidade vendida: {}'.format(str(quant_comprada)))
                print("Money atual: {}".format(str(money_atual)))
            else:
                print("HODL!")
        else:
            print("HODL!")
    if (last_decision == 1):
         money_atual = float(quant_comprada) * float(row.value_fechamento)
    decision_weight.loc[n, 'classifier'] = classifier
    decision_weight.loc[n, 'initial_maney'] = money_start
    decision_weight.loc[n, 'final_maney'] = money_atual
    decision_weight.loc[n, 'lucro_percentual'] = (money_atual - money_start)/money_start
    return decision_weight

def _apply_decisions_withperc(model_decisions, money_start, silent_mode=True):
    """
    Aplica as decisoes encontradas utilizando a ponderação dos melhores valores como método de gestão de risco
    """
    results = pd.DataFrame(columns = ['Data','initial_maney','final_maney','quant_hold','value_fechamento','perc_usado', 'action'])
    last_decision = 2
    money_atual = money_start
    quant_comprada = 0
    trades = 0
    for i, row in model_decisions.iterrows():
        if (row.real_decision != last_decision and row.real_decision !=0):
            data = row.Data
            last_decision = row.real_decision
            if (last_decision == 1):
                action='compra'
                money = float(money_atual) * float(row.perc_aplicado)
                quant_acomprar = (float(money) / float(row.value_fechamento))
                quant_comprada = quant_comprada + quant_acomprar
                money_atual = float(money_atual) - (money)
                if (silent_mode==False):
                    print("--------------------------")
                    print("Decisão de compra -")
                    print('Valor: {}'.format(str(row.value_fechamento)))
                    print('Quantidade comprada: {}'.format(str(quant_acomprar)))
                    print('Quantidade em hold: '+str(quant_comprada))
                    print("Money atual: {}".format(str(money_atual)))

            elif (last_decision == 2):
                action='venda'
                quant_vender = float(quant_comprada) * float(row.perc_aplicado)
                quant_comprada = float(quant_comprada) - float(quant_vender)
                money_comprado = float(quant_vender) * float(row.value_fechamento)
                money_atual = float(money_atual) + float(money_comprado)
                if (silent_mode==False):
                    print("--------------------------")
                    print("Decisão de venda -")
                    print('Valor: {}'.format(str(row.value_fechamento)))
                    print('Quantidade vendida: {}'.format(str(quant_vender)))
                    print('Quantidade em hold: '+str(quant_comprada))
                    print("Money atual: {}".format(str(money_atual)))
            else:
                if (silent_mode==False):
                    print("HODL!")
        else:
            if (silent_mode==False):
                print("HODL!")
        results.loc[trades, 'Data'] = data
        results.loc[trades, 'initial_maney'] = money_start
        results.loc[trades, 'final_maney'] = money_atual
        results.loc[trades, 'quant_hold'] = quant_comprada
        results.loc[trades, 'value_fechamento'] = float(row.value_fechamento)
        results.loc[trades, 'perc_usado'] = float(row.perc_aplicado)
        results.loc[trades, 'action'] = action
        trades +=1
    if (last_decision == 1 or quant_comprada > 0):
        money_atual = money_atual + (float(quant_comprada) * float(row.value_fechamento))
        quant_comprada = 0
        results.loc[trades, 'Data'] = data
        results.loc[trades, 'initial_maney'] = money_start
        results.loc[trades, 'final_maney'] = money_atual
        results.loc[trades, 'quant_hold'] = quant_comprada
        results.loc[trades, 'value_fechamento'] = float(row.value_fechamento)
        results.loc[trades, 'perc_usado'] = float(row.perc_aplicado)
        results.loc[trades, 'action'] = 'final'
    return results

def _get_model_decisions(df, classifiers, scalers, models, wait_time=5, silent_mode=True):
    """
    Aqui, utilizando os modelos encontrados:
        -verificamos quantos modelos consideram cada ação (buy, hold, sell),
        -caso todos consiredem hold, não faz nada;
        -caso a maioria seja buy/sell, compramos com a soma da ponderação encontrada a partir dos resultados anteriores
    """
    decisions = pd.DataFrame(columns=['Data','real_decision','value_fechamento','perc_aplicado'])
    # Variáveis de apoio
    trades = 0

    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)
    for i, item in df.iterrows():
        _buy = []
        _sell = []
        _hold = []
        _value_fechamento = item['Fechamento']
        for classifier in classifiers:
            row_features = scalers.get(
                classifier
            ).transform(
                pd.DataFrame(
                    item[features.get(classifier)]
                ).T
            )
            _model_decision = (models.get(classifier).predict(row_features)[0] if classifier != 'divergency_decisor' else item[classifier])
            if (_model_decision == 1):
                _buy.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
            elif (_model_decision == 2):
                _sell.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
            else:
                _hold.append(1)
        if (len(_hold) == 4):
            _model_decision = 0
            perc_ = 0
        elif (len(_buy) > len(_sell)):
            _model_decision = 1
            perc_ = sum(_buy)
            _data = item['Data']

        else:
            _model_decision = 2
            perc_ = sum(_sell)
            _data = item['Data']

        if (perc_ > 0):
            if (trades > 0):
                _last_decision_date = decisions.iloc[trades-1].Data
                _last_real_decision = decisions.iloc[trades-1].real_decision
            else:
                _last_decision_date = pd.to_datetime('01/01/1950')
                _last_real_decision = 2
            #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
            #if ((_model_decision != 0) or (_real_decision != 0)):
            if ((_data >= _last_decision_date + timedelta(days=wait_time))):
                if ((_model_decision != 0 and _model_decision != _last_real_decision)):
                # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                    new_decision = {
                        'Data':_data,
                        'real_decision':_model_decision,
                        'value_fechamento':_value_fechamento,
                        'perc_aplicado':perc_}
                    decisions = decisions.append(new_decision, ignore_index=True)
                    trades += 1
    return decisions

def _get_final_decisions(df, _stop_loss, _sell_by_stoploss, wait_time, final_scaler, final_model, classifiers, scalers, models):
    """
        -Função final com modelo de trade!
        Iremos, utilizando os suportes como stop loss, e quando encontrar um preço de fechamento 
    igual ou menor ao suporte em um ponto de suporte, a venda de x% (definido pela var _sell_by_stoploss), aplicar os modelos encontrados quando o modelo final decidir agir.
    
    """
    # Variáveis de apoio
    trades = 0
    _last_decision_date = pd.to_datetime('01/01/1950')
    _last_real_decision = 2
    decisions = pd.DataFrame(columns=['Data','real_decision','value_fechamento','perc_aplicado','stop_loss','final_model', 'holds','buys','sells'])
    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)

    for i, item in df.iterrows():
        _buy = []
        _sell = []
        _hold = []
        final_hold = 1
        _value_fechamento = item['Fechamento']
        if (item['Suporte ']==1):
            _stop_loss = item['Mínimo']
        row_features = final_scaler.transform(pd.DataFrame(item[final_features]).T)
        _model_decision = final_model.predict(row_features)[0] ## Pegamos a decisão do modelo geral
        decisao_modelo_final = final_model.predict(row_features)[0]
        if (_model_decision == 1): ## se for uma decisão de ação,
            final_hold = 0
            for classifier in classifiers: ## aplicaremos os modelos criados anteriormente
                row_features = scalers.get(
                    classifier
                ).transform(
                    pd.DataFrame(
                        item[features.get(classifier)]
                    ).T
                )
                _model_decision = (models.get(classifier).predict(row_features)[0] if classifier != 'divergency_decisor' else item[classifier])
                if (_model_decision == 1):
                    _buy.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
                elif (_model_decision == 2):
                    _sell.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
                else:
                    _hold.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
        else:
            final_hold = 1
            _model_decision = 0
            perc_ = 0
        if (decisao_modelo_final == 0): ## Usamos um comitê dos decisores: o mais votado, vira a ação! caso o modelo inicial seja hold, aplica:
            _model_decision = 0
            perc_ = 0
            _data = item['Data']
        elif (len(_buy) > len(_sell)):
            _model_decision = 1
            perc_ = sum(_buy)
            _data = item['Data']
        elif (len(_sell) > len(_buy)):
            _model_decision = 2
            perc_ = sum(_sell)
            _data = item['Data']
        else:
            _model_decision = 0
            perc_ = 0
            _data = item['Data']

        if (_model_decision > 0):
            if (trades > 0):
                _last_decision_date =  pd.to_datetime(decisions.tail(1).Data.values[0])
                _last_real_decision = decisions.tail(1).real_decision.values[0]
        #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
        #if ((_model_decision != 0) or (_real_decision != 0)):
        if ((_data >= _last_decision_date + timedelta(days=wait_time))):
            if (_value_fechamento <= _stop_loss and item['Suporte ']==1): #se encontrarmos um preço de fechamento menor ou igual ao suporte e um ponto de suporte, vendemos x%
                _model_decision = 2
                perc_ = _sell_by_stoploss
                new_decision = {
                    'Data':_data,
                    'real_decision':_model_decision,
                    'value_fechamento':_value_fechamento,
                    'perc_aplicado':perc_,
                    'stop_loss':_stop_loss,
                    'holds':len(_hold),
                    'buys':len(_buy),
                    'sells':len(_sell),
                    'final_model':decisao_modelo_final}
                decisions = decisions.append(new_decision, ignore_index=True)
                trades += 1
            elif ((_model_decision != 0 and _model_decision != _last_real_decision)):
            # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                new_decision = {
                    'Data':_data,
                    'real_decision':_model_decision,
                    'value_fechamento':_value_fechamento,
                    'perc_aplicado':perc_,
                    'stop_loss':_stop_loss,
                    'holds':len(_hold),
                    'buys':len(_buy),
                    'sells':len(_sell),
                    'final_model':decisao_modelo_final}
                decisions = decisions.append(new_decision, ignore_index=True)
                trades += 1
    return decisions

def _print_decisions(decisions):
    """
    Função auxiliar para printar as decisões
    """
    print("Data inicial: {}".format(decisions.head(1).Data.values[0]))
    print("Dinheiro inicial: R${:.2f}".format(decisions.tail(1).initial_maney.values[0]))
    print("Data final: {}".format(decisions.tail(1).Data.values[0]))
    print("Dinheiro final: R${:.2f}".format(decisions.tail(1).final_maney.values[0]))
    print("Lucro final (treino): {:.2f}%".format((decisions.tail(1).initial_maney.values[0] / decisions.tail(1).final_maney.values[0]) * 100))

In [10]:
df = pd.read_excel("./database/indicadores petrobras_fase 1_ v1.2.xlsx", sheet_name = "Tendencias").drop(['-','--'], axis=1).dropna()

In [11]:
# Model 1 - Willians
features = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','William %R']
labels = ['Willians Buy','Willians Sell']
label = 'Willians_decisor' 
df = _encode(df, labels, label)

In [12]:
class NaiveBayesClassifier():
    '''
    Bayes Theorem form
    P(y|X) = P(X|y) * P(y) / P(X)
    '''
    def calc_prior(self, features, target):
        '''
        prior probability P(y)
        calculate prior probabilities
        '''
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior
    
    def calc_statistics(self, features, target):
        '''
        calculate mean, variance for each column and convert to numpy array
        ''' 
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
              
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        '''
        calculate probability from gaussian density function (normally distributed)
        we will assume that probability of specific target value given specific class is normally distributed 
        
        probability density function derived from wikipedia:
        (1/√2pi*σ) * exp((-1/2)*((x-μ)^2)/(2*σ²)), where μ is mean, σ² is variance, σ is quare root of variance (standard deviation)
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
#         numerator = np.exp(-((x-mean)**2 / (2 * var)))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def calc_posterior(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for i in range(self.count):
            prior = np.log(self.prior[i]) ## use the log to make it more numerically stable
            conditional = np.sum(np.log(self.gaussian_density(i, x))) # use the log to make it more numerically stable
            posterior = prior + conditional
            posteriors.append(posterior)
        # return class with highest posterior probability
        return self.classes[np.argmax(posteriors)]
     

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy

    def visualize(self, y_true, y_pred, target):
        
        tr = pd.DataFrame(data=y_true, columns=[target])
        pr = pd.DataFrame(data=y_pred, columns=[target])
        
        
        fig, ax = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15,6))
        
        sns.countplot(x=target, data=tr, ax=ax[0], palette='viridis', alpha=0.7, hue=target, dodge=False)
        sns.countplot(x=target, data=pr, ax=ax[1], palette='viridis', alpha=0.7, hue=target, dodge=False)
        

        fig.suptitle('True vs Predicted Comparison', fontsize=20)

        ax[0].tick_params(labelsize=12)
        ax[1].tick_params(labelsize=12)
        ax[0].set_title("True values", fontsize=18)
        ax[1].set_title("Predicted values", fontsize=18)
        plt.show()

In [13]:
model = NaiveBayesClassifier()
X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
if (silent):
    return model, scaler
print(y_train.value_counts())
print("Accuracy (Model: {} | Gaussian Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
return model, scaler

AttributeError: 'numpy.ndarray' object has no attribute 'groupby'

In [14]:
X_train

array([[0.98529412, 0.52559301, 0.99774944, ..., 0.        , 0.        ,
        0.89310345],
       [0.97132353, 0.43539326, 1.        , ..., 0.        , 0.        ,
        0.83108108],
       [0.99485294, 0.47940075, 0.99324831, ..., 1.        , 0.        ,
        0.93918919],
       ...,
       [0.02941176, 0.59519351, 0.02025506, ..., 0.        , 0.        ,
        0.95575221],
       [0.01764706, 0.66479401, 0.00450113, ..., 0.        , 0.        ,
        0.87619048],
       [0.        , 0.47908864, 0.        , ..., 0.        , 0.        ,
        0.64761905]])